In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!wget http://images.cocodataset.org/zips/train2014.zip
!unzip -q train2014.zip
!wget http://images.cocodataset.org/zips/val2014.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!unzip -q val2014.zip
!unzip -q annotations_trainval2014.zip


! pip install 2to3
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi
!2to3 . -w
%cd PythonAPI
!python3 setup.py install

--2021-03-22 12:44:07--  http://images.cocodataset.org/zips/train2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.45.36
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.45.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510573713 (13G) [application/zip]
Saving to: ‘train2014.zip’

train2014.zip       100%[===================>]  12.58G  50.2MB/s    in 3m 25s  

2021-03-22 12:47:32 (62.8 MB/s) - ‘train2014.zip’ saved [13510573713/13510573713]

--2021-03-22 12:48:14--  http://images.cocodataset.org/zips/val2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.83.56
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.83.56|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6645013297 (6.2G) [application/zip]
Saving to: ‘val2014.zip’

val2014.zip         100%[===================>]   6.19G  43.8MB/s    in 1m 44s  

2021-03-22 12:49:59 (60.9 MB/s) - ‘val2

In [3]:
%cd /content/

/content


In [4]:
!ls "/content/drive/My Drive/

/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [5]:

from pycocotools import coco, cocoeval, _mask
from pycocotools import mask as maskUtils 
import array
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import os
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
%matplotlib inline

In [6]:
CATEGORY_NAMES=['person']

ANNOTATION_FILE_VAL = '/content/annotations/instances_val2014.json'
ANNOTATION_FILE_TRAIN = '/content/annotations/instances_train2014.json'

coco_train= coco.COCO(ANNOTATION_FILE_TRAIN)
catIds_train=coco_train.getCatIds(catNms=CATEGORY_NAMES);
imgIds_train=coco_train.getImgIds(catIds=catIds_train);
imgDict_train= coco_train.loadImgs(imgIds_train)
len(imgIds_train) , len(catIds_train)

coco_val=coco.COCO(ANNOTATION_FILE_VAL)
catIds_val=coco_val.getCatIds(catNms=CATEGORY_NAMES);
imgIds_val=coco_val.getImgIds(catIds=catIds_val);
imgDict_val= coco_val.loadImgs(imgIds_val)
len(imgIds_val) , len(catIds_val)




loading annotations into memory...
Done (t=13.68s)
creating index...
index created!
loading annotations into memory...
Done (t=5.71s)
creating index...
index created!


(21634, 1)

In [7]:
from random import shuffle

shuffle(imgIds_train)
shuffle(imgIds_val)

imgIds_train=imgIds_train[0:6000]
imgIds_val= imgIds_val[0:600]

In [8]:
train_images_person=["COCO_train2014_{0:012d}.jpg".format(ids) for ids in imgIds_train]
val_images_person=["COCO_val2014_{0:012d}.jog".format(ids) for ids in imgIds_val]

In [9]:
print(len(train_images_person), len(val_images_person))

6000 600


In [10]:
train_images_person=["COCO_train2014_{0:012d}.jpg".format(ids) for ids in imgIds_train]
print(train_images_person)
del_img_train= set(os.listdir("/content/train2014"))- set(train_images_person)
for file_name in del_img_train:
  file_name="/content/val2014/"+file_name
  if os.path.exists(file_name):
    os.remove(file_name)

['COCO_train2014_000000081721.jpg', 'COCO_train2014_000000510047.jpg', 'COCO_train2014_000000221156.jpg', 'COCO_train2014_000000098616.jpg', 'COCO_train2014_000000047256.jpg', 'COCO_train2014_000000361788.jpg', 'COCO_train2014_000000061150.jpg', 'COCO_train2014_000000157693.jpg', 'COCO_train2014_000000297964.jpg', 'COCO_train2014_000000100273.jpg', 'COCO_train2014_000000272790.jpg', 'COCO_train2014_000000556982.jpg', 'COCO_train2014_000000101211.jpg', 'COCO_train2014_000000553871.jpg', 'COCO_train2014_000000141056.jpg', 'COCO_train2014_000000473464.jpg', 'COCO_train2014_000000419019.jpg', 'COCO_train2014_000000351328.jpg', 'COCO_train2014_000000128998.jpg', 'COCO_train2014_000000559511.jpg', 'COCO_train2014_000000122631.jpg', 'COCO_train2014_000000048870.jpg', 'COCO_train2014_000000321851.jpg', 'COCO_train2014_000000170118.jpg', 'COCO_train2014_000000047804.jpg', 'COCO_train2014_000000014986.jpg', 'COCO_train2014_000000176757.jpg', 'COCO_train2014_000000293165.jpg', 'COCO_train2014_000

In [11]:
print(len(os.listdir("/content/train2014")))

12598


In [12]:
val_images_person = ["COCO_val2014_{0:012d}.jpg".format(ids) for ids in imgIds_val]
print(val_images_person)
del_img_val = set(os.listdir("/content/val2014")) - set(val_images_person)
for file_name in del_img_val:
  file_name = "/content/val2014/" + file_name
  if os.path.exists(file_name):
    os.remove(file_name)

len(os.listdir("/content/val2014"))

['COCO_val2014_000000386718.jpg', 'COCO_val2014_000000570116.jpg', 'COCO_val2014_000000570848.jpg', 'COCO_val2014_000000012731.jpg', 'COCO_val2014_000000268882.jpg', 'COCO_val2014_000000183456.jpg', 'COCO_val2014_000000306585.jpg', 'COCO_val2014_000000570781.jpg', 'COCO_val2014_000000444152.jpg', 'COCO_val2014_000000114291.jpg', 'COCO_val2014_000000257370.jpg', 'COCO_val2014_000000232625.jpg', 'COCO_val2014_000000370285.jpg', 'COCO_val2014_000000428985.jpg', 'COCO_val2014_000000001464.jpg', 'COCO_val2014_000000419867.jpg', 'COCO_val2014_000000296816.jpg', 'COCO_val2014_000000049236.jpg', 'COCO_val2014_000000380828.jpg', 'COCO_val2014_000000237487.jpg', 'COCO_val2014_000000145448.jpg', 'COCO_val2014_000000202321.jpg', 'COCO_val2014_000000369393.jpg', 'COCO_val2014_000000369791.jpg', 'COCO_val2014_000000169356.jpg', 'COCO_val2014_000000137085.jpg', 'COCO_val2014_000000563680.jpg', 'COCO_val2014_000000121503.jpg', 'COCO_val2014_000000256529.jpg', 'COCO_val2014_000000071596.jpg', 'COCO_val

600

In [13]:
from PIL import Image

In [14]:
!mkdir mask_train_2014

In [15]:
count=0

for ID in imgIds_train:
  file_path= "/content/mask_train_2014/COCO_train2014_{0:012d}.jpg".format(ID)

  sampleImgIds= coco_train.getImgIds(imgIds=[ID])
  sampleImgDict= coco_train.loadImgs(sampleImgIds[np.random.randint(0,len(sampleImgIds))])[0]

  annIds= coco_train.getAnnIds(imgIds=sampleImgDict['id'], catIds=catIds_train, iscrowd=0)   
  anns= coco_train.loadAnns(annIds)


  mask=coco_train.annToMask(anns[0])
  for i in range(len(anns)):
    mask= mask | coco_train.annToMask(anns[i])

  mask= Image.fromarray(mask *255, mode="L")
  mask.save(file_path)

  count = count +1
  print(count)



Streaming output truncated to the last 5000 lines.
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190

In [16]:
!mkdir mask_val_2014

In [17]:
count = 0 
for ID in imgIds_val:

  file_path = "/content/mask_val_2014/COCO_val2014_{0:012d}.jpg".format(ID)
  
  sampleImgIds = coco_val.getImgIds(imgIds = [ID])
  sampleImgDict = coco_val.loadImgs(sampleImgIds[np.random.randint(0,len(sampleImgIds))])[0]

  annIds = coco_val.getAnnIds(imgIds=sampleImgDict['id'], catIds=catIds_val, iscrowd=0)
  anns = coco_val.loadAnns(annIds)


  mask = coco_val.annToMask(anns[0])
  for i in range(len(anns)):
      mask = mask | coco_val.annToMask(anns[i])
  
  mask = Image.fromarray(mask * 255 , mode = "L")
  mask.save(file_path)
  
  count = count + 1
  print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [18]:
!rm -rf annotations/
!rm -rf train2014.zip
!rm -rf val2014.zip
!rm -rf annotations_trainval2014.zip


In [19]:
import os
import sys
import random

import numpy as np
import cv2
import keras
from keras.layers import * 
from keras.optimizers import *
seed= 2019

random.seed=seed
np.random.seed=seed

In [20]:
class DataGen(keras.utils.Sequence):
  
  def __init__(self, path_input, path_mask, batch_size= 8, image_size=128):

    self.ids = os.listdir(path_input)
    self.path_input=path_input
    self.path_mask=path_mask
    self.batch_size= batch_size
    self.image_size= image_size
    self.on_epoch_end()

  def __load__(self, id_name):

    image_path= os.path.join(self.path_input, id_name)
    mask_path = os.path.join(self.path_mask, id_name)

    image= cv2.imread(image_path,1)
    try:
      image= cv2.resize(image, (self.image_size, self.image_size))
    except Exception as e:
      print(str(e))
    mask= cv2.imread(mask_path, -1)
    try:
      mask= cv2.resize(mask, (self.image_size, self.image_size))
      mask= mask.reshape((self.image_size, self.image_size, 1))
    
      image= image/255.0
      mask= mask/255.0
    except Exception as e:
      print(str(e))

    return image, mask

  def __getitem__(self, index):

    if (index + 1)*self.batch_size>len(self.ids):
      self.batch_size= len(self.ids) - index * self.batch_size

    file_batch=self.ids[index * self.batch_size: (index + 1) * self.batch_size]

    images=[]
    masks=[]

    for id_name in file_batch:
      _img, _mask = self.__load__(id_name)
      images.append(_img)
      masks.append(_mask)


    images= np.array(images,dtype=np.float)
    masks = np.array(masks,dtype=np.float)

    return images, masks


  def on_epoch_end(self):
    pass

  def __len__(self):

    return int(np.ceil(len(self.ids)/ float(self.batch_size)))


In [21]:


def down_block(
    input_tensor,
    no_filters,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",
    kernel_initializer="he_normal",
    max_pool_window=(2, 2),
    max_pool_stride=(2, 2)
):
    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(input_tensor)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)

    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)

    conv = BatchNormalization(scale=True)(conv)

    # conv for skip connection
    conv = Activation("relu")(conv)

    pool = MaxPooling2D(pool_size=max_pool_window, strides=max_pool_stride)(conv)

    return conv, pool



In [22]:
def bottle_neck(
    input_tensor,
    no_filters,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",
    kernel_initializer="he_normal"
):
    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(input_tensor)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)

    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)

    return conv

In [23]:
def up_block(    
    input_tensor,
    no_filters,
    skip_connection, 
    kernel_size=(3, 3),
    strides=(1, 1),
    upsampling_factor = (2,2),
    max_pool_window = (2,2),
    padding="same",
    kernel_initializer="he_normal"):
    
    
    conv = Conv2D(
        filters = no_filters,
        kernel_size= max_pool_window,
        strides = strides,
        activation = None,
        padding = padding,
        kernel_initializer=kernel_initializer
    )(UpSampling2D(size = upsampling_factor)(input_tensor))
    
    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv) 
    
    
    conv = concatenate( [skip_connection , conv]  , axis = -1)
    
    
    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)

    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)
    
    return conv

In [24]:


def output_block(input_tensor,
    padding="same",
    kernel_initializer="he_normal"
):
    
    conv = Conv2D(
        filters=2,
        kernel_size=(3,3),
        strides=(1,1),
        activation="relu",
        padding=padding,
        kernel_initializer=kernel_initializer
    )(input_tensor)
    
    
    conv = Conv2D(
        filters=1,
        kernel_size=(1,1),
        strides=(1,1),
        activation="sigmoid",
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)
    
    
    return conv



In [25]:
def UNet(input_shape = (128,128,3)):
    
    filter_size = [64,128,256,512,1024]
    
    inputs = Input(shape = input_shape)
    
    d1 , p1 = down_block(input_tensor= inputs,
                         no_filters=filter_size[0],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal",
                         max_pool_window=(2,2),
                         max_pool_stride=(2,2))
    
    
    d2 , p2 = down_block(input_tensor= p1,
                         no_filters=filter_size[1],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal",
                         max_pool_window=(2,2),
                         max_pool_stride=(2,2))
    
    
    
    d3 , p3 = down_block(input_tensor= p2,
                         no_filters=filter_size[2],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal",
                         max_pool_window=(2,2),
                         max_pool_stride=(2,2))
    
    
    
    d4 , p4 = down_block(input_tensor= p3,
                         no_filters=filter_size[3],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal",
                         max_pool_window=(2,2),
                         max_pool_stride=(2,2))
    
    
    b = bottle_neck(input_tensor= p4,
                         no_filters=filter_size[4],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal")
    
    
    
    u4 = up_block(input_tensor = b,
                  no_filters = filter_size[3],
                  skip_connection = d4,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  upsampling_factor = (2,2),
                  max_pool_window = (2,2),
                  padding="same",
                  kernel_initializer="he_normal")
    
    u3 = up_block(input_tensor = u4,
                  no_filters = filter_size[2],
                  skip_connection = d3,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  upsampling_factor = (2,2),
                  max_pool_window = (2,2),
                  padding="same",
                  kernel_initializer="he_normal")
    
    
    u2 = up_block(input_tensor = u3,
                  no_filters = filter_size[1],
                  skip_connection = d2,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  upsampling_factor = (2,2),
                  max_pool_window = (2,2),
                  padding="same",
                  kernel_initializer="he_normal")
    
    
    u1 = up_block(input_tensor = u2,
                  no_filters = filter_size[0],
                  skip_connection = d1,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  upsampling_factor = (2,2),
                  max_pool_window = (2,2),
                  padding="same",
                  kernel_initializer="he_normal")
    
    
    
    output = output_block(input_tensor=u1 , 
                         padding = "same",
                         kernel_initializer= "he_normal")
    
    model = keras.models.Model(inputs = inputs , outputs = output)
    
    
    return model

In [33]:
model = UNet(input_shape = (128,128,3))
model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [34]:
image_size = 128 
epochs = 10
batch_size = 8

In [35]:
train_gen = DataGen(path_input = "/content/train2014" , path_mask = "/content/mask_train_2014/" , batch_size = batch_size , image_size = image_size)
val_gen = DataGen(path_input =  "/content/val2014", path_mask =  "/content/mask_val_2014", batch_size = batch_size , image_size = image_size)

train_steps =  len(os.listdir( "/content/train2014"))/batch_size


model.fit_generator(train_gen , validation_data = val_gen , steps_per_epoch = train_steps , epochs=epochs)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion faile

ValueError: ignored

In [ ]:
print len([name for name in os.listdir("/content/train2014") if os.path.isfile(name)])

In [ ]:
x, y = val_gen.__getitem__(4)
result = model.predict(x)

result = result > 0.5



fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax = fig.add_subplot(1, 2, 1)
ax.imshow(np.reshape(y[0]*255, (image_size, image_size)), cmap="gray")

ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(result[0]*255, (image_size, image_size)), cmap="gray")